In [1]:
import pandas as pd
import numpy as np
import os
import json


In [8]:
bills = []
statesDict = {}
for f in os.walk(r"D:\Big Input Data Stuff\LegiScan\input"):
    if "bill" in f[0]:
        path = f[0].split("\\")
        if int(path[-2].split("_")[0].split("-")[0]) >= 2021:
            for file in f[-1]:
                if ".json" in file:
                    fileName = f[0] + "\\" + file
                    with open(fileName, "r") as b:
                        s = json.loads(b.read())
                        bills.append(s)
                        statesDict[s['bill']['state_id']] = s["bill"]["state"]

In [9]:
statuses = [
    "N\\A",
    "Intro",
    "Engrossed",
    "Enrolled",
    "Passed",
    "Vetoed",
    "Failed",
    "Override",
    "Chaptered",
    "Refer",
    "Report Pass",
    "Report DNP",
    "Draft",
]
bill_list = []
for bill in bills:
    bill_id = bill["bill"]["bill_id"]
    status = statuses[bill["bill"]["status"]]
    bill_name = bill["bill"]["title"]
    bill_number = bill["bill"]["bill_number"]
    bill_state = bill["bill"]["state"]
    bill_desc = bill["bill"]["description"]
    text_link = (
        bill["bill"]["texts"][-1]["url"] if len(bill["bill"]["texts"]) != 0 else ""
    )
    sponsors = []
    for person in bill["bill"]["sponsors"]:
        sponsors.append(person["people_id"])
    roll_call_ids = []
    for vote in bill["bill"]["votes"]:
        roll_call_ids.append(vote["roll_call_id"])
    progresses = []
    for progress in bill["bill"]["progress"]:
        progresses.append((progress["date"], progress["event"]))
    bill_list.append(
        (
            {
                "ID": bill_id,
                "Name": bill_name,
                "Number": bill_number,
                "State": bill_state,
                "Description": bill_desc,
                "Status": status,
                "Text": text_link,
                "Progress": f"{progresses}",
                "Sponsors": f"{sponsors}",
                "Votes": f"{roll_call_ids}",
            }
        )
    )



In [10]:
bill_df = pd.DataFrame.from_records(bill_list)
bill_df.set_index("ID", inplace=True)
bill_df


,Name,Number,State,Description,Status,Text,Progress,Sponsors,Votes
ID,,,,,,,,,
1398079,Audit Of Cares Act Funds,HB1,AK,An Act providing for audits of CARES Act funds...,Intro,https://legiscan.com/AK/text/HB1/id/2232236,"[('2021-02-18', 1), ('2021-02-18', 9)]","[18610, 21262, 22861, 21266, 21272, 22860]",[]
1398054,Funter Bay Marine Park: Unangan Cemetery,HB10,AK,An Act relating to the Funter Bay marine park ...,Passed,https://legiscan.com/AK/text/HB10/id/2406357,"[('2021-02-18', 1), ('2021-02-18', 9), ('2021-...","[21261, 15011, 20423, 21262, 21271, 14389, 179...","[1049826, 1049827, 1078513, 1105641]"
1455397,Extend Workforce Invest Board Allocations,HB100,AK,An Act relating to allocations of funding for ...,Passed,https://legiscan.com/AK/text/HB100/id/2405617,"[('2021-02-18', 1), ('2021-04-21', 10), ('2021...","[17254, 21271, 14389, 6046]","[1065518, 1065519, 1079549, 1079550, 1105677, ..."
1455540,Reinsurance; Credits,HB101,AK,An Act relating to credit for reinsurance; and...,Intro,https://legiscan.com/AK/text/HB101/id/2298454,"[('2021-02-18', 1), ('2021-02-24', 9)]",[21520],[]
1455451,State Insur. Catastrophe Reserve Acct.,HB102,AK,An Act relating to the state insurance catastr...,Passed,https://legiscan.com/AK/text/HB102/id/2589146,"[('2021-02-18', 1), ('2021-02-18', 9), ('2021-...",[21520],"[1079350, 1079351, 1105552, 1204864, 1209613, ..."
...,...,...,...,...,...,...,...,...,...
1664730,Right of health care access-constitutional ame...,SJ0010,WY,A JOINT RESOLUTION proposing to amend the Wyom...,Failed,https://legiscan.com/WY/text/SJ0010/id/2646499,"[('2023-01-20', 1), ('2023-02-07', 6)]","[22676, 20193, 8711, 14226, 8675, 10602]",[1239782]
1664641,Convention of states.,SJ0011,WY,A JOINT RESOLUTION requesting Congress to call...,Engrossed,https://legiscan.com/WY/text/SJ0011/id/2646418,"[('2023-01-20', 1), ('2023-01-20', 9), ('2023-...","[24753, 23310, 24162, 23879, 18469, 23645, 234...","[1240828, 1243122, 1260993, 1263065]"
1666701,Constitutional amendment-property tax valuation.,SJ0012,WY,A JOINT RESOLUTION providing for the assessmen...,Failed,https://legiscan.com/WY/text/SJ0012/id/2648490,"[('2023-01-20', 1), ('2023-02-07', 6)]","[23643, 8679]",[1234868]


In [11]:
bill_df.to_pickle("./cleaned_input/bills_last_2_years.pkl.xz")